# Ballon d'or nominees
#### for card practice

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

---

### First, get a list of all nominees and their wiki article links

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://en.wikipedia.org/wiki/2021_Ballon_d%27Or")

In [ ]:
soup = BeautifulSoup(html, "html.parser")

In [7]:
table = soup.find_all("tbody")[1]

In [8]:
names = []
countries = []
clubs = []
pages = []
wiki_links = []
images = []

In [9]:
for row in table.find_all("tr"):
    links = row.find_all("a")
    cells = row.find_all("td")
    for link in links:
        countries.append(links[0].get("title"))
        names.append(links[1].text.strip())
        clubs.append(links[3].get("title"))
        pages.append(links[1].get("href"))

#### Then get links to wikipedia pages and thumbnails (not quite the raw image pattern...)

In [10]:
for page in pages:
    wiki_links.append("https://en.wikipedia.org/" + str(page))

In [ ]:
img_links = []
final_images = []
for wiki in wiki_links:
    pages = requests.get(str(wiki))
    soup = BeautifulSoup(pages.text, "html.parser")
    for image in soup.find_all("td", {"class": "infobox-image"}):
        links = image.find_all("a")
        img_links.append("https://en.wikipedia.org/" + str(links[0].get("href")))
        

In [ ]:
for link in img_links:
            img_page = requests.get(str(link))
            soup = BeautifulSoup(img_page.text, "html.parser")
            uploads = soup.find_all("a")
            final_images.append(uploads[9].get("href"))

In [ ]:
for source in images:
    img_page = requests.get(
        "https://en.wikipedia.org//wiki/File:AlexiaPutellas-UEFAChampionsLeaguefinal.jpg"
    )
    soup = BeautifulSoup(img_page.text, "html.parser")
    uploads = soup.find_all("a")
    final_images.append(uploads[9].get("href"))

### Put all that info in a dataframe

In [ ]:
df = pd.DataFrame(
    {
        "name": names,
        "country": countries,
        "club": clubs,
        "wikipages": wiki_links,
        "image": final_images,
    }
).drop_duplicates()

In [ ]:
df["class"] = df["country"].str.lower()

### Finally, put it in the google sheet for the rig

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [ ]:
from df2gspread import df2gspread as d2g

In [ ]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration.json", scope
)
gc = gspread.authorize(credentials)

In [ ]:
spreadsheet_key = "1UeaZYxhc8WD-U0_G8T9xLgQ6MCDIJe6uW9Xg9rrYOWA"

In [ ]:
wks_name = "data_ballon"
d2g.upload(
    df,
    spreadsheet_key,
    wks_name,
    credentials=credentials,
    row_names=False,
)

In [1]:
import us